### Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

! tar -zxvf /content/drive/MyDrive/Uni/DeepRiccy/refcocog.tar.gz
! pip3 install -qr https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt 
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

local_path = '/content/refcocog/images/'
local_annotations = '/content/refcocog/annotations/'

### PC

In [1]:
local_path = './refcocog/images/' 
local_annotations = './refcocog/annotations/' 

# Baseline

In [2]:
#imports

import json
import pickle
import torch
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
import matplotlib.pyplot as plt
import pandas as pd
from pkg_resources import packaging
import clip
import numpy as np

import os
import skimage
import IPython.display

from collections import OrderedDict
import torch

/opt/anaconda3/envs/ai/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/opt/anaconda3/envs/ai/lib/python3.10/site-packages/torchvision/image.so, 0x0006): symbol not found in flat namespace '__ZN3c106detail19maybe_wrap_dim_slowExxb'
  warn(f"Failed to load image Python extension: {e}")


## Dataset

In [3]:
class Coco(Dataset):
    def __init__(self, path_json, path_pickle, train=True):
        self.path_json = path_json
        self.path_pickle = path_pickle
        self.train = train

        #load images and annotations
        with open(self.path_json) as json_data:
            data = json.load(json_data)
            self.img_frame = pd.DataFrame(data['images'])
            self.ann_frame = pd.DataFrame(data['annotations'])

        #load annotations
        with open(self.path_pickle, 'rb') as pickle_data:
            data = pickle.load(pickle_data)
            self.refs_frame = pd.DataFrame(data)

        self.size = self.refs_frame.shape[0]

        #separate each sentence in dataframe
        self.refs_frame = self.refs_frame.explode('sent_ids')
        self.refs_frame = self.refs_frame.explode('sentences')
        self.refs_frame = self.refs_frame.reset_index(drop=True)

        self.dataset = pd.merge(self.refs_frame, self.img_frame, left_on='ann_id', right_on='id')
        self.dataset = self.dataset.drop(columns=['ann_id'])

    def __len__(self):
        return self.size
    
    def __getitem__(self, idx):
        return self.dataset.iloc[idx]


In [4]:
def split_string(string):
    string = string.split("_")
    string = string[:-1]
    string = "_".join(string)
    append = ".jpg"
    string = string + append
    
    return string

In [5]:
#test dataset

dataset = Coco(local_annotations + 'instances.json', local_annotations + "refs(umd).p")
print(dataset[0])

image_id                                                    419645
split                                                         test
sentences        {'tokens': ['there', 'is', 'red', 'colored', '...
file_name_x                 COCO_train2014_000000419645_398406.jpg
category_id                                                      8
sent_ids                                                        10
ref_id                                                           1
license                                                          2
file_name_y                        COCO_train2014_000000398406.jpg
coco_url                           http://mscoco.org/images/398406
height                                                         493
width                                                          500
date_captured                                  2013-11-14 12:11:23
flickr_url       http://farm4.staticflickr.com/3043/2503053592_...
id                                                          39

In [7]:
#test clip
clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [6]:
#define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#pass image into yolo
yolo = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
image = split_string(dataset[0]["file_name_x"])

yolo_output = yolo(local_path+image)

model, preprocess = clip.load("ViT-B/32")
model.eval()
#pass each bounding box segmented image into clip with the sentence
for i in range(len(yolo_output.xyxy[0])):

    x1 = yolo_output.xyxy[0][i][0].numpy()
    x1 = np.rint(x1)
    y1 = yolo_output.xyxy[0][i][1].numpy()
    y1 = np.rint(y1)
    x2 = yolo_output.xyxy[0][i][2].numpy()
    x2 = np.rint(x2)
    y2 = yolo_output.xyxy[0][i][3].numpy()
    y2 = np.rint(y2)

    print(x1, y1, x2, y2)

    img = Image.open(local_path+image)
    img = img.crop((x1, y1, x2, y2))
    img = preprocess(img).unsqueeze(0)
    image_imput = torch.tensor(img)
    sentence = dataset[0]["sentences"]
    text = model.tokenize([sentence]).to(device)
    with torch.no_grad():
        image_features = clip.encode_image(img).float()
        text_features = clip.encode_text(text).float()
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = text_features.cpu().numpy() @ image_features.cpu().numpy().T
    print(similarity)


Using cache found in /Users/pappol/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-4-8 Python-3.10.6 torch-2.0.0 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


RuntimeError: Couldn't load custom C++ ops. This can happen if your PyTorch and torchvision versions are incompatible, or if you had errors while compiling torchvision from source. For further information on the compatible versions, check https://github.com/pytorch/vision#installation for the compatibility matrix. Please check your PyTorch version with torch.__version__ and your torchvision version with torchvision.__version__ and verify if they are compatible, and if not please reinstall torchvision so that it matches your PyTorch install.